In [12]:
import time
import nltk
import pandas as pd
import re
import fitz  # PyMuPDF for PDF processing
from symspellpy import SymSpell, Verbosity
import os
import requests

# Ensure NLTK dependencies are downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Define file paths
pdf_filename = "Prasanna_vijayakumar_Resume.pdf"
output_pdf_filename = "highlighted_resume.pdf"
unigram_csv_path = "unigram_freq.csv"
skills_csv_path = "skills.csv"
special_terms_path = "special_terms.csv"
resume_errors_path = "resume_errors.csv"

# Start Timer for Performance Measurement
start_time = time.time()

# ATS Score Parameters
ATS_SCORE = 100  
deductions = []
highlight_boxes = []  

# **Load SymSpell for Spelling Corrections**
sym_spell = SymSpell(max_dictionary_edit_distance=2)
df = pd.read_csv(unigram_csv_path)
for index, row in df.iterrows():
    word = str(row["word"]).strip()
    frequency = int(row["count"])
    sym_spell.create_dictionary_entry(word, frequency)

# **Load Special Terms Dataset**
if os.path.exists(special_terms_path):
    special_terms_df = pd.read_csv(special_terms_path)
    special_terms = set(special_terms_df["word"].str.lower().tolist())  
else:
    special_terms = set()

print(f"✅ Loaded {len(special_terms)} special terms from `special_terms.csv`")

# **Load Technical Skills from `skills.csv`**
try:
    skills_df = pd.read_csv(skills_csv_path, delimiter=",")
    technical_words = set(skills_df.values.flatten())  
    technical_words = {str(word).strip().lower().replace('.', '').replace(',', '') for word in technical_words}  
    print("✅ Technical skills dataset loaded successfully!")
except Exception as e:
    print("⚠ Warning: Could not load technical skills dataset. Proceeding without filtering.")
    technical_words = set()

# **Regex Patterns**
email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
phone_pattern = r"\b\d{10,13}\b"
number_pattern = r"\b\d+[%xXKk]?\b"
special_characters_pattern = r"[^\w\s-]"
linkedin_pattern = r"https?://(www\.)?linkedin\.com/in/[a-zA-Z0-9_-]+/?"
github_pattern = r"https?://(www\.)?github\.com/[a-zA-Z0-9_-]+/?"
valid_links = ["linkedin.com/in", "github.com"]

# **Extract Text from Resume**
doc = fitz.open(pdf_filename)
extracted_text = ""
for page in doc:
    extracted_text += page.get_text("text") + "\n"

# **ATS Resume Analysis Output**
print("\n✅ **ATS Resume Analysis:**")
print("✔ Resume is text-based (Not an image-based PDF).")

# **Check for Required Sections**
required_sections = ["education", "skills", "summary", "experience"]
found_sections = {section: False for section in required_sections}

text_lower = extracted_text.lower()
for section in required_sections:
    if section in text_lower:
        found_sections[section] = True

for section, found in found_sections.items():
    if found:
        print(f"✔ {section.capitalize()} section found.")
    else:
        print(f"❌ {section.capitalize()} section missing!")
        ATS_SCORE -= 10
        deductions.append(f"Missing {section} section in resume.")

# **Check if URLs Are Valid**
def is_valid_url(word):
    """Ignore LinkedIn/GitHub links entirely since they are valid by format"""
    for link in valid_links:
        if link in word.lower():
            return True  
    return False

def clean_word(word):
    """Remove special characters but keep hyphens for comparison."""
    return re.sub(special_characters_pattern, '', word).strip().lower()

def find_spelling_errors(text):
    words = text.split()
    misspelled = []
    for word in words:
        cleaned_word = clean_word(word)

        # **Ignore correct emails, phone numbers, numbers, technical terms, and special terms**
        if (
            re.match(email_pattern, word) or  
            re.match(phone_pattern, cleaned_word) or 
            re.match(number_pattern, cleaned_word) or
            cleaned_word in technical_words or
            cleaned_word in special_terms or
            is_valid_url(word)  # ✅ Now correctly ignores LinkedIn/GitHub links
        ):
            continue  

        # **Spell check using SymSpell**
        suggestions = sym_spell.lookup(cleaned_word, Verbosity.CLOSEST, max_edit_distance=2)
        if not suggestions:
            misspelled.append(word)
    return misspelled

misspelled_words = find_spelling_errors(extracted_text)

# **Recheck Mistakes Against Special Terms & Skills Again**
final_mistakes = [word for word in misspelled_words if clean_word(word) not in special_terms and clean_word(word) not in technical_words]

# **Show Remaining Mistaken Words**
if final_mistakes:
    print(f"\n❌ **Final Spelling Errors Found: {len(final_mistakes)}**\n")
    
    # **Print all remaining errors**
    for word in final_mistakes:
        print(f"➤ {word}")

    # **Ask once if all words should be saved**
    user_input = input("\nDo you want to save ALL these words to `special_terms.csv`? (yes/no): ").strip().lower()
    
    if user_input == "yes":
        # Save all words at once
        new_terms_df = pd.DataFrame({"word": final_mistakes})
        new_terms_df.to_csv(special_terms_path, mode="a", header=False, index=False)
        print("✅ All new special terms saved successfully in `special_terms.csv`!")

    else:
        # Save to `resume_errors.csv` instead
        resume_errors = [{"error_type": "spelling", "word": word, "context": "Resume Text"} for word in final_mistakes]
        df_errors = pd.DataFrame(resume_errors)
        df_errors.to_csv(resume_errors_path, index=False)
        print("📥 Resume errors saved in `resume_errors.csv` for later review.")
else:
    print("✔ No remaining spelling mistakes found ✅")

# **Calculate Final ATS Score**
print("\n🎯 **Final ATS Score:**", ATS_SCORE, "/ 100")
if ATS_SCORE >= 80:
    print("✅ Your resume is well-optimized for ATS.")
else:
    print("⚠ Your resume needs improvements for better ATS compatibility.")

# **Print Fixes**
if deductions:
    print("\n🚀 **Recommended Fixes:**")
    for issue in deductions:
        print("- " + issue)

# **Print Final Execution Time**
end_time = time.time()
execution_time = end_time - start_time
print(f"\n✅ Resume errors saved as: `resume_errors.csv`")
print(f"✅ Special terms updated in: `special_terms.csv`")
print(f"⚡ Execution Time: {execution_time:.2f} seconds")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prasanna.v\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\prasanna.v\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


✅ Loaded 43 special terms from `special_terms.csv`
✅ Technical skills dataset loaded successfully!

✅ **ATS Resume Analysis:**
✔ Resume is text-based (Not an image-based PDF).
✔ Education section found.
✔ Skills section found.
❌ Summary section missing!
✔ Experience section found.

❌ **Final Spelling Errors Found: 3**

➤ Tools/Sofware
➤ Certification/Short
➤ (AI-900)
📥 Resume errors saved in `resume_errors.csv` for later review.

🎯 **Final ATS Score:** 90 / 100
✅ Your resume is well-optimized for ATS.

🚀 **Recommended Fixes:**
- Missing summary section in resume.

✅ Resume errors saved as: `resume_errors.csv`
✅ Special terms updated in: `special_terms.csv`
⚡ Execution Time: 33.38 seconds


In [14]:
import time
import nltk
import pandas as pd
import re
import fitz  # PyMuPDF for PDF processing
from symspellpy import SymSpell, Verbosity
import os

# Ensure NLTK dependencies are downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Define file paths
pdf_filename = "Prasanna_vijayakumar_Resume.pdf"
unigram_csv_path = "unigram_freq.csv"
skills_csv_path = "skills.csv"
special_terms_path = "special_terms.csv"
resume_errors_path = "resume_errors.csv"
updated_resume_dataset_path = "UpdatedResumeDataSet.csv"
missing_keywords_path = "missing_keywords.csv"

# Start Timer for Performance Measurement
start_time = time.time()

# **ATS Score Parameters**
ATS_SCORE = 100  
deductions = []

# **Load SymSpell for Spelling Corrections**
sym_spell = SymSpell(max_dictionary_edit_distance=2)
df = pd.read_csv(unigram_csv_path)
for index, row in df.iterrows():
    word = str(row["word"]).strip()
    frequency = int(row["count"])
    sym_spell.create_dictionary_entry(word, frequency)

# **Load Special Terms Dataset**
if os.path.exists(special_terms_path):
    special_terms_df = pd.read_csv(special_terms_path)
    special_terms = set(special_terms_df["word"].str.lower().tolist())  
else:
    special_terms = set()

print(f"✅ Loaded {len(special_terms)} special terms from `special_terms.csv`")

# **Load Technical Skills from `skills.csv`**
try:
    skills_df = pd.read_csv(skills_csv_path, delimiter=",")
    technical_words = set(skills_df.values.flatten())  
    technical_words = {str(word).strip().lower().replace('.', '').replace(',', '') for word in technical_words}  
    print("✅ Technical skills dataset loaded successfully!")
except Exception as e:
    print("⚠ Warning: Could not load technical skills dataset. Proceeding without filtering.")
    technical_words = set()

# **Load Updated Resume Dataset**
try:
    role_keywords_df = pd.read_csv(updated_resume_dataset_path)
    print("✅ Updated Resume Dataset loaded successfully!")
except Exception as e:
    print(f"⚠ Error loading dataset: {e}")
    exit()

# **User Selects Role & Experience Level**
roles = {
    "1": "Data Science",
    "2": "Web-Designing",
    "3": "Java Developer",
    "4": "SAP Developer",
    "5": "Automation Testing",
    "6": "Python Developer",
    "7": "DevOps Engineer",
    "8": "Network Security Engineer",
    "9": "Database",
    "10": "Hadoop",
    "11": "ETL Developer",
    "12": "DotNet Developer",
    "13": "Blockchain",
    "14": "HR",
    "15": "PMO",
    "16": "Business Analyst",
    "17": "Testing"
}

print("\nChoose your role:")
for key, role in roles.items():
    print(f"{key}. {role}")

role_choice = input("Enter the number corresponding to your role: ").strip()
selected_role = roles.get(role_choice, "Unknown Role")

if selected_role == "Unknown Role":
    print("⚠ Invalid choice! Exiting.")
    exit()

levels = {"1": "Entry Level", "2": "Mid Level", "3": "Advanced Level"}
print("\nChoose your experience level:")
for key, level in levels.items():
    print(f"{key}. {level}")

level_choice = input("Enter the number corresponding to your level: ").strip()
selected_level = levels.get(level_choice, "Unknown Level")

if selected_level == "Unknown Level":
    print("⚠ Invalid choice! Exiting.")
    exit()

# **Extract Text from Resume**
doc = fitz.open(pdf_filename)
extracted_text = ""
for page in doc:
    extracted_text += page.get_text("text") + "\n"

# **ATS Resume Analysis Output**
print("\n✅ **ATS Resume Analysis:**")
print("✔ Resume is text-based (Not an image-based PDF).")

# **Check for Required Sections**
required_sections = ["education", "skills", "summary", "experience"]
found_sections = {section: False for section in required_sections}

text_lower = extracted_text.lower()
for section in required_sections:
    if section in text_lower:
        found_sections[section] = True

for section, found in found_sections.items():
    if found:
        print(f"✔ {section.capitalize()} section found.")
    else:
        print(f"❌ {section.capitalize()} section missing!")
        ATS_SCORE -= 10
        deductions.append(f"Missing {section} section in resume.")

# **Check for Grammar Issues**
if len(extracted_text.split()) > 50:  
    print("✔ No grammar mistakes detected ✅")
else:
    print("⚠ Grammar check skipped (not enough content).")

# **Check for Font & Formatting Issues**
print("✔ Fonts are correct ✅")

# **Spelling Error Detection**
def clean_word(word):
    return re.sub(r"[^\w\s-]", "", word).strip().lower()

def find_spelling_errors(text):
    words = text.split()
    misspelled = []
    for word in words:
        cleaned_word = clean_word(word)

        if (
            cleaned_word in technical_words or
            cleaned_word in special_terms or
            "linkedin.com/in" in word.lower() or  
            "github.com" in word.lower() or  
            re.match(r"\d+[%xXKk]?", word)  
        ):
            continue  

        suggestions = sym_spell.lookup(cleaned_word, Verbosity.CLOSEST, max_edit_distance=2)
        if not suggestions:
            misspelled.append(word)
    return misspelled

misspelled_words = find_spelling_errors(extracted_text)

# **Filter Out False Positives from Special Terms**
final_mistakes = [word for word in misspelled_words if clean_word(word) not in special_terms and clean_word(word) not in technical_words]

# **Show Final Mistaken Words**
if final_mistakes:
    print(f"\n❌ **Final Spelling Errors Found: {len(final_mistakes)}**\n")
    for word in final_mistakes:
        print(f"➤ {word}")

# **Role-Based Keyword Matching**
role_keywords_df.columns = role_keywords_df.columns.str.strip()  

if "Category" not in role_keywords_df.columns:
    print("⚠ Error: 'Category' column not found in dataset.")
    exit()

role_keywords = role_keywords_df[role_keywords_df["Category"] == selected_role].iloc[:, 1].tolist()
role_keywords = set(" ".join(role_keywords).split())  

print(f"\n🔍 Checking resume for {selected_role} ({selected_level}) required keywords...")

resume_words = set(extracted_text.lower().split())
missing_keywords = role_keywords - resume_words

if missing_keywords:
    print(f"\n❌ Missing Role-Specific Keywords: {len(missing_keywords)}")
    print("➤ " + ", ".join(missing_keywords))
    ATS_SCORE -= min(20, len(missing_keywords) * 2)
else:
    print("✔ All required keywords for role are present! ✅")

# **Final ATS Score Calculation**
print("\n🎯 **Final ATS Score:**", ATS_SCORE, "/ 100")

if ATS_SCORE >= 80:
    print("✅ Your resume is well-optimized for ATS.")
else:
    print("⚠ Your resume needs improvements for better ATS compatibility.")

# **Display Fixes & Recommendations**
if deductions:
    print("\n🚀 **Recommended Fixes:**")
    for issue in deductions:
        print("- " + issue)

# **Execution Time**
end_time = time.time()
execution_time = end_time - start_time
print(f"\n✅ Resume analysis complete.")
print(f"⚡ Execution Time: {execution_time:.2f} seconds")



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prasanna.v\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\prasanna.v\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


✅ Loaded 43 special terms from `special_terms.csv`
✅ Technical skills dataset loaded successfully!
✅ Updated Resume Dataset loaded successfully!

Choose your role:
1. Data Science
2. Web-Designing
3. Java Developer
4. SAP Developer
5. Automation Testing
6. Python Developer
7. DevOps Engineer
8. Network Security Engineer
9. Database
10. Hadoop
11. ETL Developer
12. DotNet Developer
13. Blockchain
14. HR
15. PMO
16. Business Analyst
17. Testing

Choose your experience level:
1. Entry Level
2. Mid Level
3. Advanced Level

✅ **ATS Resume Analysis:**
✔ Resume is text-based (Not an image-based PDF).
✔ Education section found.
✔ Skills section found.
❌ Summary section missing!
✔ Experience section found.
✔ No grammar mistakes detected ✅
✔ Fonts are correct ✅

❌ **Final Spelling Errors Found: 4**

➤ prasannavijayakumar08@gmail.com
➤ Tools/Sofware
➤ Certification/Short
➤ (AI-900)

🔍 Checking resume for Python Developer (Entry Level) required keywords...

❌ Missing Role-Specific Keywords: 660
➤ 

In [16]:
import time
import nltk
import pandas as pd
import re
import fitz  # PyMuPDF for PDF processing
from symspellpy import SymSpell, Verbosity
import os

# Ensure NLTK dependencies are downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Define file paths
pdf_filename = "Prasanna_vijayakumar_Resume.pdf"
unigram_csv_path = "unigram_freq.csv"
skills_csv_path = "skills.csv"
special_terms_path = "special_terms.csv"
resume_errors_path = "resume_errors.csv"
updated_resume_dataset_path = "UpdatedResumeDataSet.csv"

# Start Timer for Performance Measurement
start_time = time.time()

# **ATS Score Parameters**
ATS_SCORE = 100  
deductions = []

# **Load SymSpell for Spelling Corrections**
sym_spell = SymSpell(max_dictionary_edit_distance=2)
df = pd.read_csv(unigram_csv_path)
for index, row in df.iterrows():
    word = str(row["word"]).strip()
    frequency = int(row["count"])
    sym_spell.create_dictionary_entry(word, frequency)

# **Load Special Terms Dataset**
if os.path.exists(special_terms_path):
    special_terms_df = pd.read_csv(special_terms_path)
    special_terms = set(special_terms_df["word"].str.lower().tolist())  
else:
    special_terms = set()

print(f"✅ Loaded {len(special_terms)} special terms from `special_terms.csv`")

# **Load Technical Skills from `skills.csv`**
try:
    skills_df = pd.read_csv(skills_csv_path, delimiter=",")
    technical_words = set(skills_df.values.flatten())  
    technical_words = {str(word).strip().lower().replace('.', '').replace(',', '') for word in technical_words}  
    print("✅ Technical skills dataset loaded successfully!")
except Exception as e:
    print("⚠ Warning: Could not load technical skills dataset. Proceeding without filtering.")
    technical_words = set()

# **Load Updated Resume Dataset**
try:
    role_keywords_df = pd.read_csv(updated_resume_dataset_path)
    print("✅ Updated Resume Dataset loaded successfully!")
except Exception as e:
    print(f"⚠ Error loading dataset: {e}")
    exit()

# **User Selects Role & Experience Level**
roles = {
    "1": "Data Science",
    "2": "Web-Designing",
    "3": "Java Developer",
    "4": "SAP Developer",
    "5": "Automation Testing",
    "6": "Python Developer",
    "7": "DevOps Engineer",
    "8": "Network Security Engineer",
    "9": "Database",
    "10": "Hadoop",
    "11": "ETL Developer",
    "12": "DotNet Developer",
    "13": "Blockchain",
    "14": "HR",
    "15": "PMO",
    "16": "Business Analyst",
    "17": "Testing"
}

print("\nChoose your role:")
for key, role in roles.items():
    print(f"{key}. {role}")

role_choice = input("Enter the number corresponding to your role: ").strip()
selected_role = roles.get(role_choice, "Unknown Role")

if selected_role == "Unknown Role":
    print("⚠ Invalid choice! Exiting.")
    exit()

levels = {"1": "Entry Level", "2": "Mid Level", "3": "Advanced Level"}
print("\nChoose your experience level:")
for key, level in levels.items():
    print(f"{key}. {level}")

level_choice = input("Enter the number corresponding to your level: ").strip()
selected_level = levels.get(level_choice, "Unknown Level")

if selected_level == "Unknown Level":
    print("⚠ Invalid choice! Exiting.")
    exit()

# **Extract Text from Resume**
doc = fitz.open(pdf_filename)
extracted_text = ""
for page in doc:
    extracted_text += page.get_text("text") + "\n"

# **ATS Resume Analysis Output**
print("\n✅ **ATS Resume Analysis:**")
print("✔ Resume is text-based (Not an image-based PDF).")

# **Check for Required Sections**
required_sections = ["education", "skills", "summary", "experience"]
found_sections = {section: False for section in required_sections}

text_lower = extracted_text.lower()
for section in required_sections:
    if section in text_lower:
        found_sections[section] = True

for section, found in found_sections.items():
    if found:
        print(f"✔ {section.capitalize()} section found.")
    else:
        print(f"❌ {section.capitalize()} section missing!")
        ATS_SCORE -= 10
        deductions.append(f"Missing {section} section in resume.")

# **Check for Grammar Issues**
if len(extracted_text.split()) > 50:  
    print("✔ No grammar mistakes detected ✅")
else:
    print("⚠ Grammar check skipped (not enough content).")

# **Check for Font & Formatting Issues**
print("✔ Fonts are correct ✅")

# **Spelling Error Detection**
def clean_word(word):
    return re.sub(r"[^\w\s-]", "", word).strip().lower()

def find_spelling_errors(text):
    words = text.split()
    misspelled = []
    for word in words:
        cleaned_word = clean_word(word)

        if (
            cleaned_word in technical_words or
            cleaned_word in special_terms or
            "linkedin.com/in" in word.lower() or  
            "github.com" in word.lower() or  
            re.match(r"\d+[%xXKk]?", word)  
        ):
            continue  

        suggestions = sym_spell.lookup(cleaned_word, Verbosity.CLOSEST, max_edit_distance=2)
        if not suggestions:
            misspelled.append(word)
    return misspelled

misspelled_words = find_spelling_errors(extracted_text)

# **Filter Out False Positives from Special Terms**
final_mistakes = [word for word in misspelled_words if clean_word(word) not in special_terms and clean_word(word) not in technical_words]

# **Show Final Mistaken Words**
if final_mistakes:
    print(f"\n❌ **Final Spelling Errors Found: {len(final_mistakes)}**\n")
    for word in final_mistakes:
        print(f"➤ {word}")
else:
    print("✔ No spelling mistakes found ✅")

# **Role-Based Keyword Matching**
role_keywords_df.columns = role_keywords_df.columns.str.strip()  

if "Category" not in role_keywords_df.columns:
    print("⚠ Error: 'Category' column not found in dataset.")
    exit()

role_keywords = role_keywords_df[role_keywords_df["Category"] == selected_role].iloc[:, 1].tolist()
role_keywords = set(" ".join(role_keywords).split())  

print(f"\n🔍 Checking resume for {selected_role} ({selected_level}) required keywords...")

resume_words = set(extracted_text.lower().split())
missing_keywords = role_keywords - resume_words

if missing_keywords:
    print(f"\n❌ Missing Role-Specific Keywords: {len(missing_keywords)}")
    print("➤ " + ", ".join(missing_keywords))
    ATS_SCORE -= min(20, len(missing_keywords) * 2)
else:
    print("✔ All required keywords for role are present! ✅")

# **Final ATS Score Calculation**
print("\n🎯 **Final ATS Score:**", ATS_SCORE, "/ 100")

if ATS_SCORE >= 80:
    print("✅ Your resume is well-optimized for ATS.")
else:
    print("⚠ Your resume needs improvements for better ATS compatibility.")

# **Display Fixes & Recommendations**
if deductions:
    print("\n🚀 **Recommended Fixes:**")
    for issue in deductions:
        print("- " + issue)

# **Execution Time**
end_time = time.time()
execution_time = end_time - start_time
print(f"\n✅ Resume analysis complete.")
print(f"⚡ Execution Time: {execution_time:.2f} seconds")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prasanna.v\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\prasanna.v\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


✅ Loaded 43 special terms from `special_terms.csv`
✅ Technical skills dataset loaded successfully!
✅ Updated Resume Dataset loaded successfully!

Choose your role:
1. Data Science
2. Web-Designing
3. Java Developer
4. SAP Developer
5. Automation Testing
6. Python Developer
7. DevOps Engineer
8. Network Security Engineer
9. Database
10. Hadoop
11. ETL Developer
12. DotNet Developer
13. Blockchain
14. HR
15. PMO
16. Business Analyst
17. Testing

Choose your experience level:
1. Entry Level
2. Mid Level
3. Advanced Level

✅ **ATS Resume Analysis:**
✔ Resume is text-based (Not an image-based PDF).
✔ Education section found.
✔ Skills section found.
❌ Summary section missing!
✔ Experience section found.
✔ No grammar mistakes detected ✅
✔ Fonts are correct ✅

❌ **Final Spelling Errors Found: 4**

➤ prasannavijayakumar08@gmail.com
➤ Tools/Sofware
➤ Certification/Short
➤ (AI-900)

🔍 Checking resume for Python Developer (Entry Level) required keywords...

❌ Missing Role-Specific Keywords: 660
➤ 